In [ ]:
using Revise, Oscar

# Worksheet 2: Dimension, multiplicities, etc.

## Dimension: Local and global

The definition of (Krull-) dimension in Noetherian rings is quite unaccessible in general. 
For local rings, the most practical method to compute dimensions is via the 
Hilbert-Samuel polynomial. 

In [ ]:
R, (x,y,z,w) = QQ["x", "y", "z", "w"]
M = R[x y z*(z-1); y z w]
I = ideal(R, minors(M, 2))
I

With the following command we create a *local* ordering on the variables of $R$.

In [ ]:
o = negdegrevlex(gens(R))

We can use this to produce a *standard basis* of the ideal $I$ for this ordering. 

In [ ]:
std_I = std_basis(I, o)

For a given ordering, we can also ask for the *leading ideal* of $I$ which is simply the ideal generated 
by the leading terms of the elements of the standard basis.

In [ ]:
lead_I = leading_ideal(I, ordering=o)

By construction, the leading ideal is a *homogeneous* ideal for this ordering. We can pass to the graded quotient ring for this ideal and compute the Hilbert polynomial.

In [ ]:
Rgr, _ = grade(R) # The same ring as R, but with the standard grading 
lead_Igr = ideal(Rgr, Rgr.(gens(lead_I))) # The graded version of the ideal lead_I
Q, _ = quo(Rgr, lead_Igr) # The (graded) quotient ring by that ideal
h = hilbert_polynomial(Q)

Note that the Hilbert polynomial comes with a new variable in a new ring.

In [ ]:
parent(h)

Also the Hilbert series is available 

In [ ]:
p, q = hilbert_series(Q)

Accessing the coefficients of the Hilbert polynomial is rather tedious. The reason is that 
the `coefficients` command returns an `iterator`; a common behaviour in `julia`. 

In [ ]:
coefficients(h)

How can we use such iterators? Well, we can, for example, iterate over them in `for`-loops.

In [ ]:
for c in coefficients(h)
    println(c)
end

If we do not care about such features and only want everything in a list, we can just feed the iterator to `collect`.

In [ ]:
c = collect(coefficients(h))

Or just take any other shortcut to the information of interest:

In [ ]:
last(coefficients(h))

## The Milnor number

Let us compute the Milnor number for one of the previous examples

In [ ]:
R, (x,y) = QQ["x", "y"]
f = x*y*(x+y-1)

We can implement the functionality from the lecture in various functions.

In [ ]:
function milnor_number(f::MPolyElem)
    R = parent(f)
    J = ideal(R, minors(jacobi_matrix(f), 1))
    return singular_vdim(J)
end

Note that we used a rather involved method as a black box: `singular_vdim`.

In [ ]:
milnor_number(f)

So what is happening there under the hood? 

Basically, `singular_vdim` is counting the monomials 'below' the generators of the leading ideal.
Let's see whether we can manually confirm the result.

In [ ]:
R = parent(f)
J = ideal(R, minors(jacobi_matrix(f), 1))
o = degrevlex(gens(R))
leading_ideal(J, ordering=o)

The above is the *global* milnor number. If we are only interested in singularities 
at the origin, then we can repeat the same process with a local ordering.

In [ ]:
R = parent(f)
J = ideal(R, minors(jacobi_matrix(f), 1))
o = negdegrevlex(gens(R))
leading_ideal(J, ordering=o)

We see that the singularity at the origin has Milnor number $1$ instead of the global Milnor number $4$. 
We allowed ourselves to accomodate the above commands in the following function.

In [ ]:
singular_vdim_local(J)

Similarly, we could do the same at any other rational point $p \in \mathbf A^2$, for instance at the other singular points $(0, 1)$ or $(1, 0)$ of $f$. To this end, we have to apply an isomorphism $\varphi \colon R \to R$ that takes $p$ to the origin, translate the ideal along $\varphi$ and repeat the computations for the local ordering. 
Luckily, this has already been wrapped up in the hidden (i.e. non-exported) function `Oscar.shifted_ideal` 

In [ ]:
L, _ = Localization(R, complement_of_ideal(R, [1, 0]))
J_loc = L(J)

In [ ]:
Df = jacobi_matrix(L(f))
J = ideal(L, minors(Df, 1))
J_shift = Oscar.shifted_ideal(J)

Note that `shifted_ideal` returns an ideal in the original polynomial ring $R$ and not in the localization $L$!

In [ ]:
singular_vdim_local(J_shift)

## The Tjurina number

For any singular germ $(X,0) \subset (\mathbf C,0)$ with an isolated singularity, there is the 
*Tjurina number* counting the minimal number of parameters in a versal deformation of $(X,0)$. 
For a hypersurface $X = f^{-1}(\{0\})$, $f \colon (\mathbf C^{n+1}, 0) \to (\mathbf C,0)$ it defined as the length of the *Tjurina algebra*. Let us see this in an example.

In [ ]:
R, (x,y) = QQ["x", "y"]
f0 = x^5 + y^5
J0 = ideal(R, f0) + ideal(R, minors(jacobi_matrix(f0), 1))
singular_vdim_local(J0)
# milnor_number(f0)

Note that this is equal to the Milnor number of $f$. This is due to the fact that $f$ is *quasi-homogeneous*. If the function does not have this property, the Milnor and Tjurina number can differ:

In [ ]:
f1 = x^5 + y^5 + x^3*y^3
J1 = ideal(R, f1) + ideal(R, minors(jacobi_matrix(f1), 1))
singular_vdim_local(J1)

In [ ]:
milnor_number(f1)
# J = ideal(R, minors(jacobi_matrix(f1), 1))
# primary_decomposition(J)  # Try this if you want to see what went wrong.

What a difference! But wait -- did the Milnor number really jump up that much compared to $f_0$? Somehow, it did, but *globally* and not at the origin!

In [ ]:
L, _ = Localization(R, complement_of_ideal(R, [0, 0]))
Oscar.milnor_number(L(f))

Still, we see a difference of $1$ between these two numbers.

## Milnor numbers of ICIS and the Le-Greuel formula

Recall the discussion of the Le-Greuel formula for ICIS from the lecture. 
A necessary ingredient is the construction of *random* hyperplanes. 
This can, for example, be done as follows. 

In [ ]:
[rand(QQ, -100:100) for i in 1:10]

Let us compute the Milnor number(s) for the $S_5$ singularity.

In [ ]:
R, (x,y,z) = QQ["x", "y", "z"]
I = ideal(R, [x*y, x^2 + y^2 + z^2])
# Let us first check that this is really an ICIS
@show dim(I)
@show radical(singular_slocus(I))

Next, we need to construct the isolated hypersurface singularity
given by a random linear combination of the two generators.

In [ ]:
a = [rand(QQ, -100:100) for i in 1:2]
h = sum([c*g for (c, g) in zip(a, gens(I))])
I1 = ideal(R, h)

Let us verify that this is indeed an isolated hypersurface singularity.

In [ ]:
@show dim(I1)
@show radical(singular_slocus(I1))

If we want to know more about the singularity at the origin, we may ask OSCAR (or more precisely SINGULAR in the background) to classify it.

In [ ]:
classify(h)

In [ ]:
mu1 = milnor_number(h)

Now comes the truely determinantal part of the formula.

In [ ]:
M = jacobi_matrix(gens(I))
J = ideal(R, minors(M, 2)) + I1
mu0 = singular_vdim_local(J)

Hence, the total Milnor number is $\mu(S_5) = \mu_0 - \mu_1 = 6 - 1 = 5$.

## Deformations of complete intersections

We use the $S_5$-singularity from before.

In [ ]:
R, (x,y,z) = QQ["x", "y", "z"]
I = ideal(R, [x*y, x^2 + y^2 + z^2])

The normal module is defined as $N = \mathrm{Hom}_R(I, R/I)$. OSCAR knows how to handle `Hom`-modules, 
but we need to convert all the participants into modules, first.

In [ ]:
F1 = FreeMod(R, 1) # A free module over R of rank 1
Imod, _ = sub(F1, [g*F1[1] for g in gens(I)]) # the submodule of F1 generated by I
RmodI, _ = quo(F1, Imod) # the quotient by that submodule
N, vec_to_hom = hom(Imod, RmodI)

The last command is a bit subtle. It returns a pair `(N, vec_to_hom)` consisting of the actual 
module `N` and a map `vec_to_hom`. While often we can discard any extra maps returned by 
constructors, we will need this one in the following. 

What does `vec_to_hom` do? Well, `N` is constructed as a `Hom`-module. A priori, this is just another 
finitely generated module, but also every element $v \in N$ can be interpreted as a module homomorphism 
$ I \to R/I$. The returned map `vec_to_hom` allows us to do that interpretation on the computer side 
so that we really get an honest morphism of modules.

In [ ]:
v = N[1] # the first generator of N
typeof(v)

We can try to apply `v` to an element of `Imod` directly, but this will fail.

In [ ]:
g = Imod[2] # the second generator of I
v(g) # This will complain!

Converting `v` into a homomorphism makes everything work.

In [ ]:
phi = vec_to_hom(v)
@show phi
phi(Imod[2])

We proceed with the construction of the $T^1(S_5)$. We need to divide by the image of the jacobian matrix.

In [ ]:
f = gens(I)
Df = jacobi_matrix(f)

Now we need the complicated map 
$\theta \colon R^n \to N, \quad (v_1,\dots,v_n) \mapsto \left( \sum_j a_j \cdot f_j \mapsto \sum_{i,j} v_i \frac{\partial f_j}{\partial x_i} a_j \right)$. 
For this we need the `vec_to_hom` map from before and take *preimages* of elements along this map. 
Eventually, we arrive at the following, complicated expression.

In [ ]:
theta = hom(Fn, N, [preimage(vec_to_hom, hom(Imod, RmodI, [Df[i,j]*RmodI[1] for j in 1:ngens(Imod)])) for i in 1:ngens(R)])
S, _ = image(theta)
T1, _ = quo(N, S)
kbase(T1)